# __라이브러리 로드__

In [1]:
import pandas as pd
import numpy as np
from numpy.random import RandomState
import re
import torch
from tqdm import tqdm
from konlpy.tag import Okt
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

c:\Users\user\anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#number of comments
NOC = 50
#k = 0 #start idx

## __1. 데이터 크롤링__

## __2. 데이터 전처리__

> __2.1 크롤링 데이터 로드__
- 로드 후 일단 10개만 붙이기

In [181]:
df = pd.read_excel("youtube_replies.xlsx")
#cols = list(df.keys())[1:]
df_trans = df.T[1:].reset_index()
df_trans.rename(columns={'index':'view'}, inplace=True)
df_trans.iloc[:1,:3] #example

,view,0,1
0,194914,이 교수님 정말 재미있게 강의해주시네요... 자주 뵈었음 좋겠습니다.,김기범 교수님 쓸데없는 은유나 비유없이 요점만 명확히 설명 해 주셔서 이해도 ...


In [182]:
new_df = pd.DataFrame([])
new_df["Views"] = df_trans["view"]
new_df["comments"] = df_trans.iloc[:,1] #df_trans.iloc[:,1:11] 

In [183]:
new_df = pd.DataFrame([])
new_df["Views"] = df_trans["view"]
new_df["comments"] = df_trans.iloc[:,1] #df_trans.iloc[:,1:11] 
for i in range(2,NOC+1): 
    new_df["comments"] += "\n" + df_trans.iloc[:,i]
#60만회 초과인지 기준으로 
new_df["label"] = new_df["Views"].apply(lambda x: 'Great' if int(x)>10e4*6 else 'Good')
#new_df.fillna("\n", inplace=True)
new_df = new_df.dropna() 
new_df

,Views,comments,label
0,194914,이 교수님 정말 재미있게 강의해주시네요... 자주 뵈었음 좋겠습니다.\n김기범 교수...,Good
1,319933,OCN에 켠김에 왕까지에서 허준씨를 처음 봤을때는 이렇게 될 줄은 상상도 못 했습니...,Good
3,98384,교수님 강의를 이렇게 쉽게 들을 수 있다니 행운이고 감사합니다.\n학창시절땐 과학이...,Good
4,402196,영화 <크리에이터> 예매권 이벤트 이 영상에 영화에 대한 기대평 댓글을\n남겨주신...,Good
5,150218,우와 영상 재밌게봤어요! 넘넘 차분하게 전달력 있게 말씀해주시네요.. 하도 슈뢰딩거...,Good
...,...,...,...
195,287723,다양한 천문학 이야기 전할 수 있는 좋은 기회 주신 BODA 분들께 감사드립니다! ...,Good
196,2298575,곧 안전하고 값싼 우주 여행이 실현되길 바랍니다 우주 여행 관련 이야기할 수 있는...,Great
197,362675,오늘도 초대해 주셔서 감사합니다!!!! 자주 놀러와서 축구 이야기 해볼게요!!영상 ...,Good
198,353262,"여자를 위해, 여자를 가장 잘 아는 산부인과의사언니 김지연 원장이 개발한 우먼스37...",Good


> __2.2 데이터 전처리__

In [184]:
#punctuation 제거, 특수문자 제거, lowercasing
def remove_white_space(text):
    text = re.sub(r'[\t\r\n\f\v]', ' ', str(text))
    return text

def remove_special_char(text):
    text = re.sub('[^ ㄱ-ㅣ가-힣 0-9 a-z A-Z]+', ' ', str(text))
    text.lower()
    return text

new_df.comments = new_df.comments.apply(remove_white_space)
new_df.comments = new_df.comments.apply(remove_special_char)
new_df

,Views,comments,label
0,194914,이 교수님 정말 재미있게 강의해주시네요 자주 뵈었음 좋겠습니다 김기범 교수님 ...,Good
1,319933,OCN에 켠김에 왕까지에서 허준씨를 처음 봤을때는 이렇게 될 줄은 상상도 못 했습니...,Good
3,98384,교수님 강의를 이렇게 쉽게 들을 수 있다니 행운이고 감사합니다 학창시절땐 과학이나...,Good
4,402196,영화 크리에이터 예매권 이벤트 이 영상에 영화에 대한 기대평 댓글을 남겨주신 ...,Good
5,150218,우와 영상 재밌게봤어요 넘넘 차분하게 전달력 있게 말씀해주시네요 하도 슈뢰딩거고...,Good
...,...,...,...
195,287723,다양한 천문학 이야기 전할 수 있는 좋은 기회 주신 BODA 분들께 감사드립니다 ...,Good
196,2298575,곧 안전하고 값싼 우주 여행이 실현되길 바랍니다 우주 여행 관련 이야기할 수 있는...,Great
197,362675,오늘도 초대해 주셔서 감사합니다 자주 놀러와서 축구 이야기 해볼게요 영상 재미있게...,Good
198,353262,여자를 위해 여자를 가장 잘 아는 산부인과의사언니 김지연 원장이 개발한 우먼스37...,Good


> __2.3 토크나이징 및 변수 생성__

In [185]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','ㅋㅋ']

In [186]:
okt = Okt()

tokenized_data = []
for sentence in tqdm(new_df['comments']):
    tokenized_sentence = okt.morphs(sentence, norm=True, stem=True)  # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)

100%|██████████| 198/198 [00:34<00:00,  5.73it/s]


In [187]:
tokenized_df = pd.DataFrame([])
tokenized_df["token_final"] = tokenized_data
tokenized_df

,token_final
0,"[교수, 님, 정말, 재미있다, 강의, 해주다, 자주, 뵈다, 좋다, 김기범, 교수..."
1,"[OCN, 켜다, 김, 왕, 까지에, 서다, 허준, 씨, 처음, 보다, 때, 이렇게..."
2,"[교수, 님, 강의, 이렇게, 쉬다, 들다, 수, 있다, 행운, 이고, 감사하다, ..."
3,"[영화, 크리에이터, 예매, 권, 이벤트, 영상, 영화, 대한, 기, 대, 평, 댓..."
4,"[우와, 영상, 재밌다, 보다, 넘다, 넘다, 차분하다, 전달, 력, 있다, 말씀,..."
...,...
193,"[다양하다, 천문학, 이야기, 전, 수, 있다, 좋다, 기회, 주신, BODA, 분..."
194,"[곧, 안전하다, 값싸다, 우주, 여행, 실현, 되다, 바라다, 우주, 여행, 관련..."
195,"[오늘, 초대, 주다, 감사하다, 자주, 놀러와, 서, 축구, 이야기, 해보다, 영..."
196,"[여자, 위해, 여자, 가장, 자다, 알다, 산부인과, 의사, 언니, 김지연, 원장..."


In [188]:
new_df["label"]

0       Good
1       Good
3       Good
4       Good
5       Good
       ...  
195     Good
196    Great
197     Good
198     Good
199    Great
Name: label, Length: 198, dtype: object

In [189]:
tokenized_df = pd.DataFrame([])
tokenized_df["token_final"] = tokenized_data
tokenized_df["label"] = new_df.label
tokenized_df["label"] = tokenized_df["label"].fillna('Good') 
#tokenized_df.dropna(inplace=True)
tokenized_df.head(5)

,token_final,label
0,"[교수, 님, 정말, 재미있다, 강의, 해주다, 자주, 뵈다, 좋다, 김기범, 교수...",Good
1,"[OCN, 켜다, 김, 왕, 까지에, 서다, 허준, 씨, 처음, 보다, 때, 이렇게...",Good
2,"[교수, 님, 강의, 이렇게, 쉬다, 들다, 수, 있다, 행운, 이고, 감사하다, ...",Good
3,"[영화, 크리에이터, 예매, 권, 이벤트, 영상, 영화, 대한, 기, 대, 평, 댓...",Good
4,"[우와, 영상, 재밌다, 보다, 넘다, 넘다, 차분하다, 전달, 력, 있다, 말씀,...",Good


In [190]:
### ============ 기존 코드 (비교용) ============
#df = pd.DataFrame([])
#df["token_noun"] = new_df.comments.apply(okt.nouns)
#df["token_morphs"] = new_df.comments.apply(okt.morphs)
#df["label"] = new_df.label
# ============ 저장 ================
#df.to_csv("origin_noun.csv", index=False, encoding='utf-8-sig')
#df.to_csv("origin_morphs.csv", index=False, encoding='utf-8-sig')
tokenized_df.to_csv("tokenized_df_plus.csv", index=False, encoding='utf-8-sig')

> __2.4 단어 임베딩__

In [191]:
embedding_model = Word2Vec(tokenized_df['token_final'], 
                           sg = 2, # skip-gram
                           vector_size = 100, 
                           window = 2, 
                           min_count = 1, 
                           workers = 8
                           )

print(embedding_model) 

Word2Vec<vocab=18138, vector_size=100, alpha=0.025>


In [193]:
embedding_model.wv.save_word2vec_format('week3pj_tokens_eng_w2v')
loaded_model = KeyedVectors.load_word2vec_format('week3pj_tokens_eng_w2v') # 모델 로드

model_result = loaded_model.most_similar("AI")
print(model_result)

[('방향', 0.9742190837860107), ('군대', 0.9709122180938721), ('과거', 0.9689631462097168), ('입장', 0.9677556753158569), ('생물', 0.9669126868247986), ('감정', 0.9665217399597168), ('결과', 0.9659112095832825), ('혹시', 0.9650603532791138), ('실험', 0.9631057381629944), ('경험', 0.9629468321800232)]


## __3. 모델 학습__
- 일단 테스트 모델은 임시로 Classification 문제로 정의함 

In [194]:
torch.cuda.empty_cache()

> __3.0 split data - train/val__

In [195]:
df = pd.read_csv("tokenized_df.csv")
#df = df.drop(labels="Unnamed: 0", axis=1)
df

,token_final,label
0,"['교수', '님', '정말', '재미있다', '강의', '해주다', '자주', '...",Good
1,"['켜다', '김', '왕', '까지에', '서다', '허준', '씨', '처음',...",Good
2,"['교수', '님', '강의', '이렇게', '쉬다', '들다', '수', '있다'...",Good
3,"['영화', '크리에이터', '예매', '권', '이벤트', '영상', '영화', ...",Good
4,"['우와', '영상', '재밌다', '보다', '넘다', '넘다', '차분하다', ...",Good
...,...,...
193,"['다양하다', '천문학', '이야기', '전', '수', '있다', '좋다', '...",Good
194,"['곧', '안전하다', '값싸다', '우주', '여행', '실현', '되다', '...",Great
195,"['오늘', '초대', '주다', '감사하다', '자주', '놀러와', '서', '...",Good
196,"['여자', '위해', '여자', '가장', '자다', '알다', '산부인과', '...",Great


In [196]:
rng = RandomState()

tr = df.sample(frac=0.8, random_state=rng)
val = df.loc[~df.index.isin(tr.index)]

tr.to_csv('train.csv', index=False, encoding='utf-8-sig')
val.to_csv('validation.csv', index=False, encoding='utf-8-sig')

> __3.1 실험 설계__

In [197]:
import torchtext
from torchtext.legacy.data import Field #from torchtext.data import Field

def tokenizer(text):
    text = re.sub('[\[\]\']', '', str(text))
    text = text.split(', ')
    return text

TEXT = Field(tokenize=tokenizer)
LABEL = Field(sequential = False)

In [198]:
from torchtext.legacy.data import TabularDataset #from torchtext.data import TabularDataset 

train, validation = TabularDataset.splits(
    path = './',
    train = 'train.csv',
    validation = 'validation.csv',
    format = 'csv',
    fields = [('text', TEXT), ('label', LABEL)],
    skip_header = True
)

print("Train:", train[0].text,  train[0].label)
print("Validation:", validation[0].text, validation[0].label)

Train: ['이번', '역다', '우리', '은하', '철도', '종착역', '인', '안드로메다', '안드로메다', '이다', '우주먼지', '님', '차다', '설명', '을', '알차다', '많다', '우주', '유튜버들', '을', '보고', '있다', '한데', '어렵다', '내용', '을', '편하다', '말씀', '능력', '월등', '해', '요', '우주먼지', '지웅', '배', '님', '을', '알', '게', '되다', '전', '까지는', '정도', '우주', '지식', '을', '갖다', '있다', '사람', '해외', '에만', '존재', '알다', '정말', '너무', '너무', '신기하다', '알다', '아', '는걸', '설명', '정말', '다른', '차원', '이란', '걸', '또', '느끼다', '설명', '을', '전부', '다', '알다', '내용', '인데', '흥미롭다', '보다', '정말', '쉬다', '이해', '좋다', '설명', '해주다', '감사하다', '오다', '도플러', '효과', '대해', '이렇게', '쉬다', '설명', '해주다', '감동', '이네', '요', '이렇게', '어렵다', '이야기', '이렇다', '쉬다', '설명', '얼마나', '연구', '하고', '공부', '그', '덕', '을', '제', '보다', '넘다', '감사하다', '정말로', '쉬다', '설명', '을', '명확하다', '너무나', '자다', '설명', '주다', '사실', '어떻다', '사실', '을', '알', '아', '들다', '없다', '어렵다', '말로', '설명', '사람', '진정하다', '전문가', '아니다', '왜냐면', '본인', '정확하다', '이해', '못', '때문', '액면', '그대로', '설명', '보다', '어렵다', '설명', '겁니다', '대한민국', '천문학', '이런', '분', '의하다', '밝다', '생각', '안드로메다', '존재', '외계', '문', '명도',

In [199]:
import torch
from torchtext.vocab import Vectors
from torchtext.legacy.data import BucketIterator #from torchtext.data import BucketIterator 

vectors = Vectors(name="week3pj_tokens_w2v")

TEXT.build_vocab(train, vectors = vectors, min_freq = 1, max_size = None)
LABEL.build_vocab(train)

vocab = TEXT.vocab

device = "cpu" #torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_iter, validation_iter = BucketIterator.splits(
    datasets = (train, validation),
    batch_size = 8,
    device = device,
    sort = False
)

print('임베딩 벡터의 개수와 차원 : {} '.format(TEXT.vocab.vectors.shape))

임베딩 벡터의 개수와 차원 : torch.Size([15495, 100]) 


> __3.2 TextCNN 모델링__

In [200]:
import torch.nn as nn   
import torch.optim as optim 
import torch.nn.functional as F 

class TextCNN(nn.Module): 
    
    def __init__(self, vocab_built, emb_dim, dim_channel, kernel_wins, num_class):
        
        super(TextCNN, self).__init__()
        
        self.embed = nn.Embedding(len(vocab_built), emb_dim)
        self.embed.weight.data.copy_(vocab_built.vectors)      
    
        self.convs = nn.ModuleList([nn.Conv2d(1, dim_channel, (w, emb_dim)) for w in kernel_wins])
        self.relu = nn.ReLU()                
        self.dropout = nn.Dropout(0.4)         
        self.fc = nn.Linear(len(kernel_wins)*dim_channel, num_class)     
        
    def forward(self, x):  
      
        emb_x = self.embed(x)           
        emb_x = emb_x.unsqueeze(1)  

        con_x = [self.relu(conv(emb_x)) for conv in self.convs]       

        pool_x = [F.max_pool1d(x.squeeze(-1), x.size()[2]) for x in con_x]    
        
        fc_x = torch.cat(pool_x, dim=1) 
        fc_x = fc_x.squeeze(-1)       
        fc_x = self.dropout(fc_x)         

        logit = self.fc(fc_x)     
        
        return logit

> __3.3 모델 train/eavluate 함수 정의__

In [201]:
def train(model, device, train_itr, optimizer):
    
    model.train()                               
    corrects, train_loss = 0.0,0        
    
    for batch in train_itr:
        
        text, target = batch.text, batch.label      
        text = torch.transpose(text, 0, 1)          
        target.data.sub_(1)                                 
        text, target = text.to(device), target.to(device)  

        optimizer.zero_grad()                           
        logit = model(text)                         
    
        loss = F.cross_entropy(logit, target)   
        loss.backward()  
        optimizer.step()  
        
        train_loss += loss.item()    
        result = torch.max(logit,1)[1] 
        corrects += (result.view(target.size()).data == target.data).sum()
        
    train_loss /= len(train_itr.dataset)
    accuracy = 100.0 * corrects / len(train_itr.dataset)

    return train_loss, accuracy

In [202]:
def evaluate(model, device, itr):
    
    model.eval()
    corrects, test_loss = 0.0, 0

    for batch in itr:
        
        text = batch.text
        target = batch.label
        
        text = torch.transpose(text, 0, 1)
        target.data.sub_(1)
        text, target = text.to(device), target.to(device)
        
        logit = model(text)
        loss = F.cross_entropy(logit, target)
        #print(target,logit)
        test_loss += loss.item()
        result = torch.max(logit,1)[1]
        corrects += (result.view(target.size()).data == target.data).sum()

    test_loss /= len(itr.dataset) 
    accuracy = 100.0 * corrects / len(itr.dataset)
    
    return test_loss, accuracy

> __3.4 모델 학습 및 성능 확인__

In [203]:
EPOCH = 15

In [204]:
model = TextCNN(vocab, 100, 10, [1,1,1], 2).to(device)
print(model)

device = "cpu" #torch.device("cuda" if torch.cuda.is_available() else "cpu")

optimizer = optim.Adam(model.parameters(), lr=0.001)

best_test_acc = -1

for epoch in range(1, EPOCH+1):
 
    tr_loss, tr_acc = train(model, device, train_iter, optimizer) 
    print('Train Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, tr_loss, tr_acc))
    
    val_loss, val_acc = evaluate(model, device, validation_iter)
    print('Valid Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, val_loss, val_acc))
        
    if val_acc > best_test_acc:
        best_test_acc = val_acc
        
        print("model saves at {} accuracy".format(best_test_acc))
        torch.save(model.state_dict(), "TextCNN_Best_Validation")
    
    print('-----------------------------------------------------------------------------')

TextCNN(
  (embed): Embedding(15495, 100)
  (convs): ModuleList(
    (0): Conv2d(1, 10, kernel_size=(1, 100), stride=(1, 1))
    (1): Conv2d(1, 10, kernel_size=(1, 100), stride=(1, 1))
    (2): Conv2d(1, 10, kernel_size=(1, 100), stride=(1, 1))
  )
  (relu): ReLU()
  (dropout): Dropout(p=0.4, inplace=False)
  (fc): Linear(in_features=30, out_features=2, bias=True)
)
Train Epoch: 1 	 Loss: 0.08567318244825435 	 Accuracy: 60.75949478149414%
Valid Epoch: 1 	 Loss: 0.07931894212961196 	 Accuracy: 70.0%
model saves at 70.0 accuracy
-----------------------------------------------------------------------------
Train Epoch: 2 	 Loss: 0.08351742164998115 	 Accuracy: 63.2911376953125%
Valid Epoch: 2 	 Loss: 0.07806072384119034 	 Accuracy: 70.0%
-----------------------------------------------------------------------------
Train Epoch: 3 	 Loss: 0.08287845966936666 	 Accuracy: 63.2911376953125%
Valid Epoch: 3 	 Loss: 0.07766415625810623 	 Accuracy: 70.0%
-------------------------------------------